In [9]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
import os

# Load the pre-trained emotion detection model
emotion_model = load_model("face_emotion.h5")

# Create a list of emotion labels
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Load a face detection classifier (you can replace it with your preferred face detector)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create an ImageDataGenerator object with the same preprocessing parameters as your training data
train_aug = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=25, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, 
        zoom_range=0.2,horizontal_flip=True, 
        fill_mode="nearest")

# Get the current directory
current_dir = os.getcwd()

# Construct the path to the image
image_path = os.path.join(current_dir, '..', 'uploads/photo-1699243507318.jpg')

frame = cv2.imread(image_path)

frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

# Convert the frame to grayscale for face detection
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect faces in the frame
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

for (x, y, w, h) in faces:
    # Extract the face from the frame
    face = gray[y:y+h, x:x+w]

    # Convert grayscale face to 3-channels
    face = cv2.cvtColor(face, cv2.COLOR_GRAY2BGR)
    
    # Resize the face image to the size expected by the model
    face = cv2.resize(face, (48, 48))

    # Preprocess the face image using the ImageDataGenerator object
    face = train_aug.random_transform(face)
    face = np.expand_dims(face, axis=0)
    face = preprocess_input(face)  # Now it should not raise an error

    # Reshape the face image for the model input
    face = face.reshape((1, 48, 48, 3))

    # Predict the emotion label for the face
    emotion = emotion_model.predict(face)
    emotion_label = emotion_labels[np.argmax(emotion)]

    # Draw a rectangle around the detected face and annotate with the emotion label
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

print(emotion_label)


1/1 [==============================] - 0s 200ms/step
Surprise
